# Agente Inteligente para la Generación Automática de Queries MongoDB a partir de Lenguaje Natural

Este notebook demuestra el uso de un agente inteligente capaz de transformar instrucciones en lenguaje natural en queries MongoDB, incluyendo soporte para joins dinámicos.

In [1]:
# Si tienes problemas de compatibilidad, ejecuta también esta celda para forzar la versión correcta de pydantic-core.
!pip install --force-reinstall --no-deps pydantic_core==2.41.1
!pip install --upgrade pydantic openai

  Using cached pydantic_core-2.41.1-cp311-cp311-win_amd64.whl.metadata (7.4 kB)
Using cached pydantic_core-2.41.1-cp311-cp311-win_amd64.whl (2.0 MB)


  Using cached pydantic-2.12.0-py3-none-any.whl.metadata (83 kB)
Using cached pydantic-2.12.0-py3-none-any.whl (459 kB)


In [2]:
import sys
sys.path.append('..')
sys.path.append('../src')
import importlib
import AgenteGeneradorQueryMongo
importlib.reload(AgenteGeneradorQueryMongo)
from AgenteGeneradorQueryMongo import SmartMongoQueryGenerator
from dataset_manager import create_default_dataset
manager = create_default_dataset()
generator = SmartMongoQueryGenerator(dataset_manager=manager)
# Ejemplo de instrucción en lenguaje natural para JOIN (proyección dinámica de campos del join)
instruccion_join = "une la colección empleados con la colección departamentos usando el campo departamento_id proyecta los campos nombre, apellido, departamentos_info, departamento_nombre"
query_join = generator.generate_query('empleados', instruccion_join)
print('Query generada para join:')
print(query_join)

Query generada para join:
[
  {
    "$lookup": {
      "from": "departamentos",
      "localField": "departamento_id",
      "foreignField": "departamento_id",
      "as": "departamentos_info"
    }
  },
  {
    "$unwind": "$departamentos_info"
  },
  {
    "$project": {
      "nombre": 1,
      "apellido": 1,
      "departamentos_info": 1,
      "departamento_nombre": 1
    }
  }
]


Puedes modificar la variable `instruccion` para probar otras frases en lenguaje natural, incluyendo joins con diferentes nombres de colección y campo, o instrucciones que no requieran join.

In [3]:
# Ejemplo 1: filtra registros cuyo Devices.ServicePoints.ShipOutCycles.Transactions.Total sea mayor a 3000
instruccion = "filtra registros cuyo Total sea mayor a 3000"
query = generator.generate_query('transacciones', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
# Mostrar el pipeline como dict para depuración
import re
pipeline_match = re.search(r'aggregate\\((\[.*\])\\)', query, re.DOTALL)
if pipeline_match:
    print("\nPipeline generado:")
    print(pipeline_match.group(1))

[DEBUG] Pipeline generado: {'$match': {'Devices.ServicePoints.ShipOutCycles.Transactions.Total': {'$gt': 3000.0}}}
Instrucción: filtra registros cuyo Total sea mayor a 3000
Query generado:
[
  {
    "$match": {
      "Devices.ServicePoints.ShipOutCycles.Transactions.Total": {
        "$gt": 3000.0
      }
    }
  }
]


In [4]:
# Ejemplo 2: muestra los nombres y apellidos de los empleados del departamento de ventas
instruccion = "muestra los nombres y apellidos de los empleados del departamento de ventas"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: muestra los nombres y apellidos de los empleados del departamento de ventas
Query generado:
[
  {
    "$match": {
      "departamento": "ventas"
    }
  },
  {
    "$project": {
      "apellidos": 1,
      "nombres": 1,
      "ventas": 1,
      "departamento": 1,
      "del": 1
    }
  }
]



In [5]:

# Ejemplo 3: cuenta cuántos empleados hay en cada departamento
instruccion = "cuenta cuántos empleados hay en cada departamento"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: cuenta cuántos empleados hay en cada departamento
Query generado:
[
  {
    "$group": {
      "_id": "$departamento",
      "count": {
        "$sum": 1
      }
    }
  },
  {
    "$project": {
      "departamento": "$_id",
      "count": 1,
      "_id": 0
    }
  }
]



In [6]:

# Ejemplo 4: ordena los empleados por fecha de ingreso descendente
instruccion = "ordena los empleados por fecha de ingreso descendente"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: ordena los empleados por fecha de ingreso descendente
Query generado:
[
  {
    "$sort": {
      "fechadeingreso": -1
    }
  }
]



In [7]:

# Ejemplo 5: busca empleados cuyo nombre comience con 'A'
instruccion = "busca empleados cuyo nombre comience con 'A'"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: busca empleados cuyo nombre comience con 'A'
Query generado:
[
  {
    "$match": {
      "nombre": {
        "$regex": "^A",
        "$options": "i"
      }
    }
  }
]



In [8]:
# Ejemplo 6: filtra transacciones realizadas entre 2023-01-01 y 2023-01-31
instruccion = "filtra transacciones realizadas entre 2023-01-01 y 2023-01-31"
query = generator.generate_query('transacciones', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: filtra transacciones realizadas entre 2023-01-01 y 2023-01-31
Query generado:
[
  {
    "$match": {
      "Date": {
        "$gte": "2023-01-01",
        "$lte": "2023-01-31"
      }
    }
  },
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles.Transactions"
  }
]



In [9]:
# Ejemplo 7: muestra los 5 productos más vendidos
instruccion = "muestra los 5 productos más vendidos"
query = generator.generate_query('productos', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: muestra los 5 productos más vendidos
Query generado:
[
  {
    "$group": {
      "_id": "$nombre",
      "total_vendidos": {
        "$sum": "$ventas"
      }
    }
  },
  {
    "$sort": {
      "total_vendidos": -1
    }
  },
  {
    "$limit": 5
  },
  {
    "$project": {
      "producto": "$_id",
      "total_vendidos": 1,
      "_id": 0
    }
  }
]



In [10]:
# Ejemplo 8: agrega la suma total de ventas por mes
instruccion = "agrega la suma total de ventas por mes"
query = generator.generate_query('ventas', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: agrega la suma total de ventas por mes
Query generado:
[
  {
    "$addFields": {
      "anio_mes": {
        "$substr": [
          "$Date",
          0,
          7
        ]
      }
    }
  },
  {
    "$group": {
      "_id": "$anio_mes",
      "suma_total_ventas": {
        "$sum": "$total"
      }
    }
  },
  {
    "$sort": {
      "_id": 1
    }
  },
  {
    "$project": {
      "mes": "$_id",
      "suma_total_ventas": 1,
      "_id": 0
    }
  }
]



In [11]:
# Ejemplo 9: une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta
instruccion = "une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta"
query = generator.generate_query('ventas', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta
Query generado:
[
  {
    "$lookup": {
      "from": "clientes",
      "localField": "cliente_id",
      "foreignField": "cliente_id",
      "as": "clientes_info"
    }
  },
  {
    "$unwind": "$clientes_info"
  },
  {
    "$project": {
      "nombre_cliente": 1,
      "total_venta": 1
    }
  }
]



In [12]:


#Ejemplo 10: filtra clientes que no hayan realizado compras en el último año
instruccion = "filtra clientes que no hayan realizado compras en el penultimo año"
query = generator.generate_query('clientes', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()




Instrucción: filtra clientes que no hayan realizado compras en el penultimo año
Query generado:
[
  {
    "$match": {
      "compras": {
        "$not": {
          "$elemMatch": {
            "Date": {
              "$gte": "2023-10-09",
              "$lt": "2024-10-08"
            }
          }
        }
      }
    }
  }
]



In [13]:
instruccion = "une la colección empleados con la colección departamentos usando el campo departamento_id"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: une la colección empleados con la colección departamentos usando el campo departamento_id
Query generado:
[
  {
    "$lookup": {
      "from": "departamentos",
      "localField": "departamento_id",
      "foreignField": "departamento_id",
      "as": "departamentos_info"
    }
  },
  {
    "$unwind": "$departamentos_info"
  }
]



In [14]:
# Caso 1: Header simple
instruccion_header = """
crear campo dateMascara que convierta el campo Date a formato %Y%m%d usando los primeros 19 caracteres
crear campo reg que concatene: "1", "002", el campo Date convertido a formato %Y%m%d%H%M%S usando los primeros 19 caracteres, "00", "01", un espacio, un salto de línea, otro salto de línea
"""
print('--- Caso 1: Header ---')
query_header = generator.generate_query('header', instruccion_header)
print(query_header)

--- Caso 1: Header ---
[
  {
    "$project": {
      "_id": 0,
      "dateMascara": {
        "$dateToString": {
          "date": {
            "$dateFromString": {
              "dateString": {
                "$substr": [
                  "$Date",
                  0,
                  19
                ]
              }
            }
          },
          "format": "%Y%m%d"
        }
      },
      "reg": {
        "$concat": [
          "1",
          "002",
          {
            "$dateToString": {
              "date": {
                "$dateFromString": {
                  "dateString": {
                    "$substr": [
                      "$Date",
                      0,
                      19
                    ]
                  }
                }
              },
              "format": "%Y%m%d%H%M%S"
            }
          },
          "00",
          "01",
          " ",
          "\n",
          "\n"
        ]
      }
    }
  }
]


# Ejemplos avanzados de instrucciones en lenguaje natural
A continuación se muestran tres casos de uso para probar el agente generador de queries MongoDB:

In [15]:
# Caso 2: Detalle avanzado
instruccion_detalle = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles
desanidar Devices.ServicePoints.ShipOutCycles.Transactions
agrupar por date, deviceId, branchCode, subChannelCode, shipOutCode, currencyCode, confirmationCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total
proyectar los caracteres de la posición 2 en adelante del deviceId
crear campo confirmationCode que sea _id.confirmationCode o " " si es nulo
crear campo totalParteEntera que sea el primer elemento del split del total por punto
crear campo totalParteDecimal que sea el segundo elemento del split del total por punto o "00" si es nulo
crear campo reg que sea la concatenación de "5", la condición de moneda, la fecha, "00", el deviceId con padding, el shipOutCode con padding, la condición de sucursal, el monto con padding y el código de confirmación con padding
ordenar por deviceId y shipOutCode y subChannelCode y currencyCode
"""
print('--- Caso 2: Detalle ---')
query_detalle = generator.generate_query('detalle', instruccion_detalle)
print(query_detalle)

--- Caso 2: Detalle ---
[
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles.Transactions"
  },
  {
    "$group": {
      "_id": {
        "Date": "$Date",
        "Id": "$Devices.Id",
        "branchCode": "$Devices.BranchCode",
        "subChannelCode": "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        "shipOutCode": "$Devices.ServicePoints.ShipOutCycles.Code",
        "currencyCode": "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      "total": {
        "$sum": "$total"
      }
    }
  },
  {
    "$addFields": {
      "reg": {
        "$concat": [
          "5",
          "la condición de moneda",
          "la fecha",
          "00",
          "el deviceId con padding",
          "el shipOutCode con padding",
          "la condición de sucursal",
          "el monto con padding",
          "

In [16]:
# Caso 3: Detalle con agrupaciones y padding
instruccion_detalle2 = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles con preserveNullAndEmptyArrays
desanidar Devices.ServicePoints.ShipOutCycles.Transactions con preserveNullAndEmptyArrays
agrupar por deviceId, branchCode, subChannelCode, shipOutCode, currencyCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total en soles y en dólares según el código de moneda
luego agrupa todo y suma totalSoles y totalDolares, y cuenta total de registros en soles y en dólares según el código de moneda
crear campo totalParteEnteraSoles que sea el primer elemento del split de totalSoles por punto
crear campo totalParteDecimalSoles que sea el segundo elemento del split de totalSoles por punto o "00" si es nulo
crear campo totalParteEnteraDolares que sea el primer elemento del split de totalDolares por punto
crear campo totalParteDecimalDolares que sea el segundo elemento del split de totalDolares por punto o "00" si es nulo
crear campo reg que concatene: "9", el total de registros con padding, el total de registros en soles con padding, el total de registros en dólares con padding, el monto en soles con padding, el monto en dólares con padding, un salto de línea, otro salto de línea
"""
print('--- Caso 3: Detalle agrupado ---')
query_detalle2 = generator.generate_query('detalle2', instruccion_detalle2)
print(query_detalle2)

--- Caso 3: Detalle agrupado ---
[
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": {
      "path": "$Devices.ServicePoints.ShipOutCycles",
      "preserveNullAndEmptyArrays": true
    }
  },
  {
    "$unwind": {
      "path": "$Devices.ServicePoints.ShipOutCycles.Transactions",
      "preserveNullAndEmptyArrays": true
    }
  },
  {
    "$group": {
      "_id": {
        "Id": "$Devices.Id",
        "branchCode": "$Devices.BranchCode",
        "subChannelCode": "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        "shipOutCode": "$Devices.ServicePoints.ShipOutCycles.Code",
        "currencyCode": "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      "totalSoles": {
        "$sum": {
          "$cond": [
            {
              "$eq": [
                "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode",
                "PEN"
              ]
            },
            "$Devices.Ser